In [1]:
import pandas as pd

In [9]:
train=pd.read_csv('data/fraudTrain.csv',index_col=None)
test=pd.read_csv('data/fraudTest.csv',index_col=None)

In [10]:
## we will use 23 months for training and 1 month for testing 
## we will have some static feats
## we will have some dynamic feats for each time shots
##

In [11]:
total_data=pd.concat((train,test))
total_data.head(1)

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0


In [12]:
total_data.trans_date_trans_time.max(),total_data.trans_date_trans_time.min()

('2020-12-31 23:59:34', '2019-01-01 00:00:18')

In [17]:
def get_discrete_time(time):
    year=int(time[:4])
    month=int(time[5:7])
    if year>2019:
        month+=12
    return month

In [18]:
total_data['discrete_time']=total_data['trans_date_trans_time'].apply(lambda x: get_discrete_time(x) )

In [19]:
total_data.discrete_time.max(),total_data.discrete_time.min()

(24, 1)

In [24]:
##transductive setting 

train_data=total_data[total_data['discrete_time']<24]
test_data=total_data[total_data['discrete_time']==24]
train_data.cc_num.nunique(),test_data.cc_num.nunique(),total_data.cc_num.nunique()

(997, 910, 999)

In [25]:
train_data.merchant.nunique(),test_data.merchant.nunique(),total_data.merchant.nunique()

(693, 693, 693)

In [26]:
test_data=test_data[test_data['cc_num'].isin(train_data.cc_num.unique())]

In [41]:
cc_num_dict=dict((j,i) for i,j in enumerate(train_data.cc_num.unique()))
merchant_dict=dict((j,i) for i,j in enumerate(train_data.merchant.unique()))
gender_dict=dict((j,i) for i,j in enumerate(train_data.gender.unique()))
city_dict=dict((j,i) for i,j in enumerate(train_data.city.unique()))
state_dict=dict((j,i) for i,j in enumerate(train_data.state.unique()))
street_dict=dict((j,i) for i,j in enumerate(train_data.street.unique()))
zip_dict=dict((j,i) for i,j in enumerate(train_data.zip.unique()))
merchant_cat_dict=dict((j,i) for i,j in enumerate(train_data.category.unique()))
job_dict=dict((j,i) for i,j in enumerate(train_data.job.unique()))


In [42]:
feat_cc_num_pre_df=train_data[['cc_num','gender','city','state','lat','long','zip',
'street','job','dob','city_pop'
]]

In [43]:
feat_cc_num_pre_df=feat_cc_num_pre_df.drop_duplicates()

In [49]:
feat_cc_num_pre_df['cc_id']=feat_cc_num_pre_df['cc_num'].apply(lambda x: cc_num_dict[x])
feat_cc_num_pre_df['state_id']=feat_cc_num_pre_df['state'].apply(lambda x: state_dict[x])
feat_cc_num_pre_df['city_id']=feat_cc_num_pre_df['city'].apply(lambda x: city_dict[x])
feat_cc_num_pre_df['street_id']=feat_cc_num_pre_df['street'].apply(lambda x: street_dict[x])
feat_cc_num_pre_df['job_id']=feat_cc_num_pre_df['job'].apply(lambda x: job_dict[x])
feat_cc_num_pre_df['zip_id']=feat_cc_num_pre_df['zip'].apply(lambda x: zip_dict[x])
feat_cc_num_pre_df['dob_year']=feat_cc_num_pre_df['dob'].apply(lambda x: x[:4])

In [47]:
total_data.columns

Index([u'Unnamed: 0', u'trans_date_trans_time', u'cc_num', u'merchant',
       u'category', u'amt', u'first', u'last', u'gender', u'street', u'city',
       u'state', u'zip', u'lat', u'long', u'city_pop', u'job', u'dob',
       u'trans_num', u'unix_time', u'merch_lat', u'merch_long', u'is_fraud',
       u'discrete_time'],
      dtype='object')

In [50]:
cc_num_static_feat=feat_cc_num_pre_df[['cc_id','state_id','city_id','street_id','job_id','dob_year','city_pop','zip_id','lat','long']]

In [52]:
cc_num_static_feat.head(1)

,cc_id,state_id,city_id,street_id,job_id,dob_year,city_pop,zip_id,lat,long
0,0,0,0,0,0,1988,3495,0,36.0788,-81.1781


In [61]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore')
encoder_df = pd.DataFrame(encoder.fit_transform(cc_num_static_feat[['state_id','job_id']]).toarray())

In [62]:
encoder_df

,0,1,2,3,4,5,6,7,8,9,...,538,539,540,541,542,543,544,545,546,547
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
